In [1]:
#pip install azure.ai.ml

In [33]:
# Check core SDK version number
#import azureml.core

#print("SDK version:", azureml.core.VERSION)

In [1]:
# Import required libraries
from azure.identity import DefaultAzureCredential
from azure.identity import AzureCliCredential
from azure.ai.ml import automl, Input, MLClient

from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.automl import (
    classification,
    ClassificationPrimaryMetrics,
    ClassificationModels,
)

In [2]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

credential = DefaultAzureCredential()
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your AzureML workspace
    subscription_id = "80ef7369-572a-4abd-b09a-033367f44858"
    resource_group = "mlopsdev"
    workspace = "mlopswk"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)

We could not find config.json in: . or in its parent directories. 


In [3]:
workspace = ml_client.workspaces.get(name=ml_client.workspace_name)

subscription_id = ml_client.connections._subscription_id
resource_group = workspace.resource_group
workspace_name = ml_client.workspace_name

output = {}
output["Workspace"] = workspace_name
output["Subscription ID"] = subscription_id
output["Resource Group"] = resource_group
output["Location"] = workspace.location
output

{'Workspace': 'mlopswk',
 'Subscription ID': '80ef7369-572a-4abd-b09a-033367f44858',
 'Resource Group': 'mlopsdev',
 'Location': 'centralus'}

In [4]:
my_training_data_input = Input(
    type=AssetTypes.MLTABLE, path="./data/"
)
my_training_data_test = Input(
    type=AssetTypes.MLTABLE, path="./data/"
)
my_training_data_validate = Input(
    type=AssetTypes.MLTABLE, path="./data/"
)

In [7]:
# General job parameters
compute_name = "cpu-cluster"
max_trials = 5
exp_name = "automlv2-Titanic-experiment"

In [8]:
# Create the AutoML classification job with the related factory-function.

classification_job = automl.classification(
    compute=compute_name,
    experiment_name=exp_name,
    training_data=my_training_data_input,
    target_column_name="Survived",
    primary_metric="accuracy",
    n_cross_validations=5,
    enable_model_explainability=True,
    tags={"my_custom_tag": "Titanic Training"},
)

# Limits are all optional
classification_job.set_limits(
    timeout_minutes=600,
    trial_timeout_minutes=20,
    max_trials=max_trials,
    # max_concurrent_trials = 4,
    # max_cores_per_trial: -1,
    enable_early_termination=True,
)

# Training properties are optional
classification_job.set_training(
    blocked_training_algorithms=[ClassificationModels.LOGISTIC_REGRESSION],
    enable_onnx_compatible_models=True,
)

In [9]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(
    classification_job
)  # submit the job to the backend

print(f"Created job: {returned_job}")

Created job: ClassificationJob({'log_verbosity': <LogVerbosity.INFO: 'Info'>, 'task_type': <TaskType.CLASSIFICATION: 'Classification'>, 'environment_id': None, 'environment_variables': None, 'outputs': {}, 'type': 'automl', 'status': 'NotStarted', 'log_files': None, 'name': 'gray_room_6c34vq59cg', 'description': None, 'tags': {'my_custom_tag': 'Titanic Training'}, 'properties': {}, 'id': '/subscriptions/80ef7369-572a-4abd-b09a-033367f44858/resourceGroups/mlopsdev/providers/Microsoft.MachineLearningServices/workspaces/mlopswk/jobs/gray_room_6c34vq59cg', 'base_path': './', 'creation_context': <azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.SystemData object at 0x7f70e457f970>, 'serialize': <msrest.serialization.Serializer object at 0x7f70e47aec20>, 'inputs': {}, 'display_name': 'gray_room_6c34vq59cg', 'experiment_name': 'automlv2-Titanic-experiment', 'compute': 'cpu-cluster', 'services': {'Tracking': <azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.JobServi

In [10]:
ml_client.jobs.stream(returned_job.name)

RunId: gray_room_6c34vq59cg
Web View: https://ml.azure.com/runs/gray_room_6c34vq59cg?wsid=/subscriptions/80ef7369-572a-4abd-b09a-033367f44858/resourcegroups/mlopsdev/workspaces/mlopswk

Execution Summary
RunId: gray_room_6c34vq59cg
Web View: https://ml.azure.com/runs/gray_room_6c34vq59cg?wsid=/subscriptions/80ef7369-572a-4abd-b09a-033367f44858/resourcegroups/mlopsdev/workspaces/mlopswk



In [11]:
#pip install azureml-mlflow

In [12]:
#pip install mlflow

In [13]:
import mlflow

# Obtain the tracking URL from MLClient
MLFLOW_TRACKING_URI = ml_client.workspaces.get(
    name=ml_client.workspace_name
).mlflow_tracking_uri

print(MLFLOW_TRACKING_URI)

azureml://centralus.api.azureml.ms/mlflow/v1.0/subscriptions/80ef7369-572a-4abd-b09a-033367f44858/resourceGroups/mlopsdev/providers/Microsoft.MachineLearningServices/workspaces/mlopswk


In [14]:
# Set the MLFLOW TRACKING URI

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

print("\nCurrent tracking uri: {}".format(mlflow.get_tracking_uri()))


Current tracking uri: azureml://centralus.api.azureml.ms/mlflow/v1.0/subscriptions/80ef7369-572a-4abd-b09a-033367f44858/resourceGroups/mlopsdev/providers/Microsoft.MachineLearningServices/workspaces/mlopswk


In [15]:
from mlflow.tracking.client import MlflowClient

# Initialize MLFlow client
mlflow_client = MlflowClient()

In [16]:
job_name = returned_job.name

# Example if providing an specific Job name/ID
# job_name = "b4e95546-0aa1-448e-9ad6-002e3207b4fc"

# Get the parent run
mlflow_parent_run = mlflow_client.get_run(job_name)

print("Parent Run: ")
print(mlflow_parent_run)

Parent Run: 
<Run: data=<RunData: metrics={'AUC_macro': 0.8613727478202907,
 'AUC_micro': 0.8745165942050773,
 'AUC_weighted': 0.8613727478202907,
 'accuracy': 0.8226539451384094,
 'average_precision_score_macro': 0.8553008492780101,
 'average_precision_score_micro': 0.8608517595994416,
 'average_precision_score_weighted': 0.8638819407540405,
 'balanced_accuracy': 0.8044171585938884,
 'f1_score_macro': 0.8066070205664607,
 'f1_score_micro': 0.8226539451384094,
 'f1_score_weighted': 0.8205446171410469,
 'log_loss': 0.44147799144337424,
 'matthews_correlation': 0.6224430509362389,
 'norm_macro_recall': 0.6088343171877767,
 'precision_score_macro': 0.8187248531719586,
 'precision_score_micro': 0.8226539451384094,
 'precision_score_weighted': 0.8274815981356782,
 'recall_score_macro': 0.8044171585938884,
 'recall_score_micro': 0.8226539451384094,
 'recall_score_weighted': 0.8226539451384094,
 'weighted_accuracy': 0.8386363915718051}, params={}, tags={'_azureml.ComputeTargetType': 'STANDARD

In [17]:
# Get the best model's child run

best_child_run_id = mlflow_parent_run.data.tags["automl_best_child_run_id"]
print("Found best child run id: ", best_child_run_id)

best_run = mlflow_client.get_run(best_child_run_id)

print("Best child run: ")
print(best_run)

Found best child run id:  gray_room_6c34vq59cg_4
Best child run: 
<Run: data=<RunData: metrics={'AUC_macro': 0.8613727478202907,
 'AUC_micro': 0.8745165942050773,
 'AUC_weighted': 0.8613727478202907,
 'accuracy': 0.8226539451384094,
 'average_precision_score_macro': 0.8553008492780101,
 'average_precision_score_micro': 0.8608517595994416,
 'average_precision_score_weighted': 0.8638819407540405,
 'balanced_accuracy': 0.8044171585938884,
 'f1_score_macro': 0.8066070205664607,
 'f1_score_micro': 0.8226539451384094,
 'f1_score_weighted': 0.8205446171410469,
 'log_loss': 0.44147799144337424,
 'matthews_correlation': 0.6224430509362389,
 'norm_macro_recall': 0.6088343171877767,
 'precision_score_macro': 0.8187248531719586,
 'precision_score_micro': 0.8226539451384094,
 'precision_score_weighted': 0.8274815981356782,
 'recall_score_macro': 0.8044171585938884,
 'recall_score_micro': 0.8226539451384094,
 'recall_score_weighted': 0.8226539451384094,
 'weighted_accuracy': 0.8386363915718051}, par

In [18]:
best_run.data.metrics

{'f1_score_micro': 0.8226539451384094,
 'recall_score_weighted': 0.8226539451384094,
 'average_precision_score_micro': 0.8608517595994416,
 'precision_score_weighted': 0.8274815981356782,
 'recall_score_macro': 0.8044171585938884,
 'precision_score_macro': 0.8187248531719586,
 'norm_macro_recall': 0.6088343171877767,
 'average_precision_score_macro': 0.8553008492780101,
 'AUC_macro': 0.8613727478202907,
 'recall_score_micro': 0.8226539451384094,
 'f1_score_macro': 0.8066070205664607,
 'AUC_micro': 0.8745165942050773,
 'average_precision_score_weighted': 0.8638819407540405,
 'accuracy': 0.8226539451384094,
 'precision_score_micro': 0.8226539451384094,
 'log_loss': 0.44147799144337424,
 'f1_score_weighted': 0.8205446171410469,
 'AUC_weighted': 0.8613727478202907,
 'balanced_accuracy': 0.8044171585938884,
 'matthews_correlation': 0.6224430509362389,
 'weighted_accuracy': 0.8386363915718051}

In [19]:
import os

# Create local folder
local_dir = "./artifact_downloads"
if not os.path.exists(local_dir):
    os.mkdir(local_dir)

In [20]:
# Download run's artifacts/outputs
local_path = mlflow_client.download_artifacts(
    best_run.info.run_id, "outputs", local_dir
)
print("Artifacts downloaded in: {}".format(local_path))
print("Artifacts: {}".format(os.listdir(local_path)))

Artifacts downloaded in: /mnt/batch/tasks/shared/LS_root/mounts/clusters/devbox2022/code/Users/babal/AMLBook2022/AMLBook2022/Chapter5/artifact_downloads/outputs
Artifacts: ['conda_env_v_1_0_0.yml', 'engineered_feature_names.json', 'env_dependencies.json', 'featurization_summary.json', 'generated_code', 'internal_cross_validated_models.pkl', 'mlflow-model', 'model.onnx', 'model.pkl', 'model_onnx.json', 'pipeline_graph.json', 'run_id.txt', 'scoring_file_pbi_v_1_0_0.py', 'scoring_file_v_1_0_0.py', 'scoring_file_v_2_0_0.py']


In [21]:
# Show the contents of the MLFlow model folder
os.listdir("./artifact_downloads/outputs/mlflow-model")

['conda.yaml', 'MLmodel', 'model.pkl', 'python_env.yaml', 'requirements.txt']

In [22]:
# import required libraries
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
    ProbeSettings,
)
from azure.ai.ml.constants import ModelType

In [23]:
# Creating a unique endpoint name with current datetime to avoid conflicts
import datetime

online_endpoint_name = "titanic-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is a sample online endpoint for mlflow model",
    auth_mode="key",
    tags={"foo": "bar"},
)
ml_client.begin_create_or_update(endpoint)

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://titanic-11141948894167.centralus.inference.ml.azure.com/score', 'swagger_uri': 'https://titanic-11141948894167.centralus.inference.ml.azure.com/swagger.json', 'name': 'titanic-11141948894167', 'description': 'this is a sample online endpoint for mlflow model', 'tags': {'foo': 'bar'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/80ef7369-572a-4abd-b09a-033367f44858/resourcegroups/mlopsdev/providers/microsoft.machinelearningservices/workspaces/mlopswk/onlineendpoints/titanic-11141948894167', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/80ef7369-572a-4abd-b09a-033367f44858/providers/Microsoft.MachineLearningServices/locations/centralus/mfeOperationsStatus/oe:0ac61d30-2265-42dd-9c0a-245ddb042f62:114adb6d-22ae-48f6-8010-164221db92b6?api-version=2022-02-01-preview'}, 'id': '/subscriptions/80ef7369-572a-4abd-b09a-033367f44858/resourceGrou

In [ ]:
#ml_client.begin_create_or_update(endpoint)

In [24]:
model_name = "titanic-model"
model = Model(
    path=f"azureml://jobs/{best_run.info.run_id}/outputs/artifacts/outputs/model.pkl",
    name=model_name,
    description="my sample mlflow model",
)

# for downloaded file
# model = Model(path="artifact_downloads/outputs/model.pkl", name=model_name)

registered_model = ml_client.models.create_or_update(model)
registered_model.id

'/subscriptions/80ef7369-572a-4abd-b09a-033367f44858/resourceGroups/mlopsdev/providers/Microsoft.MachineLearningServices/workspaces/mlopswk/models/titanic-model/versions/7'

In [25]:
env = Environment(
    name="automl-tabular-env",
    description="environment for automl inference",
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210727.v1",
    conda_file="artifact_downloads/outputs/conda_env_v_1_0_0.yml",
)

In [26]:
code_configuration = CodeConfiguration(
    code="artifact_downloads/outputs/", scoring_script="scoring_file_v_2_0_0.py"
)

In [27]:
deployment = ManagedOnlineDeployment(
    name="titanic-deploy",
    endpoint_name=online_endpoint_name,
    model=registered_model.id,
    environment=env,
    code_configuration=code_configuration,
    instance_type="Standard_DS2_V2",
    instance_count=1,
)

In [28]:
ml_client.online_deployments.begin_create_or_update(deployment)

.......................................................................

In [29]:
# bankmarketing deployment to take 100% traffic
endpoint.traffic = {"titanic-deploy": 100}
#ml_client.begin_create_or_update(endpoint)

In [30]:
# test the blue deployment with some sample data
ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="titanic-deploy",
    request_file="sample-titanic-data.json",
)

'{"Results": [0]}'

In [31]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

{'titanic-deploy': 0}
https://titanic-11141948894167.centralus.inference.ml.azure.com/score


In [ ]:
ml_client.online_deployments.get_logs(
    name=online_endpoint_name, endpoint_name=online_endpoint_name, lines=50
)

In [32]:
ml_client.online_endpoints.begin_delete(name=online_endpoint_name)

............................................................................................